# Image sharpness and finding the focus point

This notebook illustrates how to find the focus point of an imaging detector by computing the image sharpness as a function of camera position.

In [ ]:
import scipp as sc
from ess import imaging as img
from ess.imaging import data
import scippnexus as sx
import numpy as np

## Load the data

We load the image data, and attach the camera stage position to the data array.

In [ ]:
dg = sx.load(data.get_tbl_orca_focussing_path())
da = dg["entry"]["instrument"]["orca_detector"]["data"]
da.coords["position"] = dg["entry"]["instrument"]["camera_stage"]["position_setpoint"][
    "value"
].data
da = da.rename_dims(time="position")
da

The data contains 61 images, one for each camera position.

We can plot the first image in the sequence to get an impression of what it looks like:

In [ ]:
da["position", 0].plot(aspect="equal")

## Computing image sharpness

The `ess.imaging.tools` module provides tools for image manipulation.
The `sharpness` function computes image sharpness over two spatial dimensions (that need to be specified).
The remaining `position` dimension will remain in the output.

In [ ]:
sharp = img.tools.sharpness(da, dims=["dim_1", "dim_2"])
sharp

A simple plot of the sharpness reveals a 'best position' where the sharpness peaks (around 180 mm):

In [ ]:
fig = sharp.plot()
fig

## Find the sharpest image

To accurately find the sharpest image, and the associated camera stage position with the best focus,
we filter outliers by applying a 75th percentile threshold to the data.
We then compute the mean position weighted by the sharpness on the remaining points.

In [ ]:
threshold = sc.scalar(np.percentile(sharp.values, 75))
sel = sharp.data >= threshold
subset = sharp[sel]
subset

In [ ]:
# Compute weighted mean of selected points
pos = (subset.coords["position"] * subset.data).sum() / subset.data.sum()
pos

In [ ]:
fig.ax.axhline(threshold.value, color="lightgray", label="threshold")
fig.ax.axvline(pos.value, color="k", label="focus point")
fig.ax.legend()
fig

## Image comparison

Finally, we can quickly compare the first image in the stack to the sharpest image determined above.

In [ ]:
# Show a smaller region to highlight details
dx = 100
xslice = ("dim_2", slice(363, 363 + dx))
yslice = ("dim_1", slice(90, 90 + dx))

# Find the closest position to the focus point
closest = subset.coords["position"][
    np.argmin(sc.abs(subset.coords["position"] - pos).values)
]

da["position", 0][xslice][yslice].plot(aspect="equal", title="First image") + da[
    "position", closest
][xslice][yslice].plot(aspect="equal", title="Sharpest image")